<a href="https://colab.research.google.com/github/xKDR/Shedding-light-on-the-Russia-Ukraine-war/blob/main/reproducible_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WORK IN PROGRESS

In [20]:
using Pkg
Pkg.add(["GeoJSON", "NighttimeLights", "Rasters", "CSV", "Dates", "DataFrames"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
  17718.1 ms  ? NighttimeLights
  18654.4 ms  ? Rasters → RastersMakieExt


In [66]:
;git clone https://github.com/xKDR/Shedding-light-on-the-Russia-Ukraine-war.git

Cloning into 'Shedding-light-on-the-Russia-Ukraine-war'...
Updating files: 100% (272/272), done.


In [69]:
## Fetching the raster files
## This will take around 5 mins

run(`apt-get install git-lfs -y`)
run(`git lfs install`)
cd("Shedding-light-on-the-Russia-Ukraine-war")
run(`git lfs pull`)

Reading package lists...

LoadError: failed process: Process(`[4mapt-get[24m [4minstall[24m [4mgit-lfs[24m [4m-y[24m`, ProcessSignaled(2)) [0]


In [21]:
using GeoJSON
using GeoDataFrames
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [174]:
# Define path constants
const FIGURES_DIRECTORY = "figures"
const TABLES_DIRECTORY = "tables"
const VECTOR_DIRECTORY = "data/vector"
const TABULAR_DIRECTORY = "data/tabular"
const RADIANCE_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"
const CFOBS_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

"/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

In [100]:
const START_DATE = Date(2014, 1)
const END_DATE = Date(2016, 1)

2016-01-01

In [24]:
function read_vector(filename)
# Use GeoJSON.read, which expects an IO
    geo = open(filename) do f
        GeoJSON.read(f)
    end

    # Now extract features
    features = geo.features

    # Build DataFrame with properties + geometry
    return DataFrame([merge(Dict(f.properties), Dict(:geometry => f.geometry)) for f in features])
end

read_vector (generic function with 1 method)

In [25]:
russia_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

Row,geometry,NAME_1,NL_NAME_1,GID_0,VARNAME_1,HASC_1,ISO_1,path,TYPE_1,GID_1,COUNTRY,CC_1,ENGTYPE_1,layer
,Abstract…,String,String,String,String,String,String,String,String,String,String,String,String,String
1,2D Polygon,Chukot1,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
2,2D Polygon,Chukot2_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
3,2D Polygon,Chukot3,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
4,2D Polygon,Chukot4,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
5,2D Polygon,Chukot5_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
6,2D Polygon,Krasnoyarsk1,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
7,2D Polygon,Krasnoyarsk2,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
8,2D MultiPolygon,Krasnoyarsk3,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
9,2D Polygon,Krasnoyarsk4,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states


In [26]:
function safe_centre_of_mass(slice::AbstractArray, state_name::String)
    s = sum(skipmissing(slice))
    if ismissing(s) || isnan(s) || s == 0.0f0
        return [NaN, NaN]
    end
    try
        return NighttimeLights.centre_of_mass(slice)
    catch e
        if isa(e, InexactError)
            println("Warning: CoM calculation failed for '$state_name'. Returning [NaN, NaN].")
            return [NaN, NaN]
        else
            rethrow(e)
        end
    end
end

safe_centre_of_mass (generic function with 1 method)

In [172]:
import Base.Filesystem: basename

"""
Extracts Date from filename.
Matches either `YYYY-MM-DD` or `YYYY-MM-DDTHH:MM:SS`.
"""
function file_to_date(f::AbstractString)
    b = basename(f)
    m = match(r"\d{4}-\d{2}-\d{2}(?:T\d{2}:\d{2}:\d{2})?", b)
    m === nothing && error("Cannot parse date from filename: $f")
    s = m.match
    if occursin('T', s)
        return Date(DateTime(s, dateformat"yyyy-mm-ddTHH:MM:SS"))
    else
        return Date(s, dateformat"yyyy-mm-dd")
    end
end


"""
Reads rasters between `start_date` and `end_date`
and returns cropped datacubes for radiance and cloud-free observations.
"""
function read_region_as_datacubes(region; start_date::Date, end_date::Date,
    radiance_path=RADIANCE_RASTER_DIRECTORY, cf_path=CFOBS_RASTER_DIRECTORY)

    # all .tif files
    rad_all = sort(joinpath.(radiance_path, filter(f -> endswith(f, ".tif"), readdir(radiance_path))))
    cf_all  = sort(joinpath.(cf_path, filter(f -> endswith(f, ".tif"), readdir(cf_path))))


    # restrict to date window
    rad_inwin = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), rad_all)
    cf_inwin  = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), cf_all)

    # sort by extracted date
    rad_inwin = rad_inwin[sortperm(file_to_date.(rad_inwin))]
    cf_inwin  = cf_inwin[sortperm(file_to_date.(cf_inwin))]

    # build raster lists
# build raster lists, now fully in memory
    rad_rasters = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in rad_inwin]
    cf_rasters  = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in cf_inwin]


    # raster series with actual dates
    rad_series = RasterSeries(rad_rasters, Ti(file_to_date.(rad_inwin)))
    cf_series  = RasterSeries(cf_rasters,  Ti(file_to_date.(cf_inwin)))

    # convert to datacubes
    rad_datacube = Rasters.combine(rad_series, Ti)
    cf_datacube  = Rasters.combine(cf_series, Ti)
    return rad_datacube, cf_datacube
end


read_region_as_datacubes

In [175]:
radiance, cfobs = read_region_as_datacubes(
    russia_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

(Float32[1.61 1.63 … 0.51 0.54; 1.57 1.64 … 0.55 0.54; … ; 1.67 1.77 … 0.42 0.31; 1.75 1.74 … 0.41 0.38;;; 2.13 2.04 … 0.43 0.46; 2.09 2.09 … 0.44 0.45; … ; 1.66 1.53 … 0.45 0.49; 1.61 1.63 … 0.38 0.38;;; 1.11 0.93 … 0.13 0.11; 1.06 1.05 … 0.1 0.15; … ; 1.51 1.54 … 0.18 0.12; 1.52 1.54 … 0.19 0.21;;; … ;;; 2.46 2.45 … 0.39 0.43; 2.46 2.49 … 0.46 0.49; … ; 1.4 1.34 … 0.31 0.14; 1.29 1.41 … 0.22 0.21;;; 1.99 2.03 … 0.29 0.29; 2.03 2.1 … 0.28 0.33; … ; 2.42 2.45 … 0.02 0.08; 2.46 2.42 … 0.03 0.07;;; 2.01 1.96 … 0.45 0.43; 1.99 1.99 … 0.43 0.41; … ; 2.72 2.56 … 0.62 0.56; 2.83 2.67 … 0.48 0.63], UInt16[0x003e 0x003d … 0x001c 0x001a; 0x0040 0x003e … 0x001b 0x001c; … ; 0x003a 0x003a … 0x0005 0x0004; 0x003a 0x003b … 0x0004 0x0005;;; 0x0020 0x001f … 0x0018 0x0017; 0x0021 0x001f … 0x0018 0x0017; … ; 0x001e 0x0021 … 0x000b 0x000c; 0x001e 0x001f … 0x000a 0x000c;;; 0x001b 0x0019 … 0x001a 0x0018; 0x001b 0x001b … 0x001a 0x0019; … ; 0x001d 0x001d … 0x001a 0x001a; 0x001d 0x001d … 0x001a 0x001b;;; … ;;

In [176]:
radiance, cfobs = read_region_as_datacubes(russia_state_split[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

(Float32[1.61 1.63 … 0.51 0.54; 1.57 1.64 … 0.55 0.54; … ; 1.67 1.77 … 0.42 0.31; 1.75 1.74 … 0.41 0.38;;; 2.13 2.04 … 0.43 0.46; 2.09 2.09 … 0.44 0.45; … ; 1.66 1.53 … 0.45 0.49; 1.61 1.63 … 0.38 0.38;;; 1.11 0.93 … 0.13 0.11; 1.06 1.05 … 0.1 0.15; … ; 1.51 1.54 … 0.18 0.12; 1.52 1.54 … 0.19 0.21;;; … ;;; 2.46 2.45 … 0.39 0.43; 2.46 2.49 … 0.46 0.49; … ; 1.4 1.34 … 0.31 0.14; 1.29 1.41 … 0.22 0.21;;; 1.99 2.03 … 0.29 0.29; 2.03 2.1 … 0.28 0.33; … ; 2.42 2.45 … 0.02 0.08; 2.46 2.42 … 0.03 0.07;;; 2.01 1.96 … 0.45 0.43; 1.99 1.99 … 0.43 0.41; … ; 2.72 2.56 … 0.62 0.56; 2.83 2.67 … 0.48 0.63], UInt16[0x003e 0x003d … 0x001c 0x001a; 0x0040 0x003e … 0x001b 0x001c; … ; 0x003a 0x003a … 0x0005 0x0004; 0x003a 0x003b … 0x0004 0x0005;;; 0x0020 0x001f … 0x0018 0x0017; 0x0021 0x001f … 0x0018 0x0017; … ; 0x001e 0x0021 … 0x000b 0x000c; 0x001e 0x001f … 0x000a 0x000c;;; 0x001b 0x0019 … 0x001a 0x0018; 0x001b 0x001b … 0x001a 0x0019; … ; 0x001d 0x001d … 0x001a 0x001a; 0x001d 0x001d … 0x001a 0x001b;;; … ;;

In [177]:
function process_state(state_row::DataFrameRow)
    state_name = state_row.NAME_1
    state_geometry = state_row.geometry
    @show state_name
    @show state_geometry
    radiance, cfobs = read_region_as_datacubes(state_geometry;
        start_date=START_DATE, end_date=END_DATE,
        radiance_path=RADIANCE_RASTER_DIRECTORY,
        cf_path=CFOBS_RASTER_DIRECTORY)


    print("file read correctly")

    radiance = clean_complete(radiance, cfobs; bgthreshold=4)
    cfobs = nothing # Release memory

    radiance = Raster(Float32.(radiance); missingval=missing)
    radiance[radiance .> 100.0f0] .= 0.0f0 # Cap high radiance values
    radiance = mask(radiance; with=state_geometry)

    dates = Date.(dims(radiance, Ti).val)
    num_timesteps = length(dates)

    if num_timesteps == 0
        println("Warning: No valid data for '$state_name' after cleaning.")
        return DataFrame()
    end

    agg_radiance = [sum(skipmissing(view(radiance, Ti(i)))) for i in 1:num_timesteps]
    com = [safe_centre_of_mass(view(radiance, Ti(i)), state_name) for i in 1:num_timesteps]

    return DataFrame(; dates, aggregate_radiance=agg_radiance, com, state_name)
end

process_state (generic function with 1 method)

In [ ]:
all_states_df = DataFrame()

for i in 1:nrow(russia_state_split)
    state_row = russia_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(russia_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(all_states_df, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(all_states_df)
    println("Aggregating results...")
    grouped = groupby(all_states_df, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


┌ Warning: outlier_variance failed, using all pixels as stable
│   exception =
│    ArgumentError: empty data vector
│    Stacktrace:
│      [1] _quantilesort!(v::Vector{Union{}}, sorted::Bool, minp::Float64, maxp::Float64)
│        @ Statistics ~/.julia/packages/Statistics/gbcbG/src/Statistics.jl:985
│      [2] quantile!(v::Vector{Union{}}, p::Float64; sorted::Bool, alpha::Float64, beta::Float64)
│        @ Statistics ~/.julia/packages/Statistics/gbcbG/src/Statistics.jl:980
│      [3] quantile(itr::Base.SkipMissing{Vector{Missing}}, p::Float64; sorted::Bool, alpha::Float64, beta::Float64)
│        @ Statistics ~/.julia/packages/Statistics/gbcbG/src/Statistics.jl:1090
│      [4] outlier_variance(radiance_datacube::Raster{Missing, 3, Tuple{X{Projected{Float64, LinRange{Float64, Int64}, DimensionalData.Dimensions.Lookups.ForwardOrdered, DimensionalData.Dimensions.Lookups.Regular{Float64}, DimensionalData.Dimensions.Lookups.Intervals{DimensionalData.Dimensions.Lookups.Start}, DimensionalD